<a href="https://colab.research.google.com/github/egarmir/TFM_LastMile_Pulse/blob/main/LastMile%20Pulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
import pandas as pd
import joblib
import os
import plotly.express as px
import folium
from folium.plugins import HeatMap

# --- 1. CARGA DE MODELOS ---
ruta_drive = '/content/drive/MyDrive/GRADIO - LastMilePulse/'
clf_model = joblib.load(os.path.join(ruta_drive, 'pipeline_clasificacion_lade.joblib'))
reg_model = joblib.load(os.path.join(ruta_drive, 'pipeline_regresion_lade.joblib'))

# --- 2. FUNCIONES ---

# A) Función Predicción Individual
def predecir_manual(hour, distance, orders, aoi, day_name, mean_delay, volume):
    try:
        dias_dict = {"Lunes": 0, "Martes": 1, "Miércoles": 2, "Jueves": 3, "Viernes": 4, "Sábado": 5, "Domingo": 6}
        day_week_num = dias_dict[day_name]

        datos = {
            'hour': hour, 'distance_km': distance, 'orders_by_courier': orders,
            'aoi_type': aoi, 'day_of_week': day_week_num,
            'courier_mean_delay': mean_delay, 'courier_volume': volume,
            'courier_sla_rate': 0.85,
            'aoi_congestion': 0.5 if (13 <= hour <= 21) else 0.2,
            'is_weekend': 1 if day_week_num >= 5 else 0,
            'is_peak_hour': 1 if (13 <= hour <= 15) or (20 <= hour <= 22) else 0
        }

        df = pd.DataFrame([datos])
        cols_modelo = ['aoi_type', 'courier_sla_rate', 'is_weekend', 'orders_by_courier',
                       'day_of_week', 'aoi_congestion', 'is_peak_hour', 'courier_mean_delay',
                       'courier_volume', 'distance_km', 'hour']

        prob = clf_model.predict_proba(df[cols_modelo])[0, 1]
        minutos = reg_model.predict(df[cols_modelo])[0]

        estado = "🔴 RIESGO DE RETRASO" if (prob >= 0.34 or minutos >= 90) else "🟢 SIN RIESGO"

        return estado, f"{prob:.1%}", f"{round(minutos, 1)} min"

    except Exception as e:
        return f"Error: {e}", "---", "---"

# B) Función Análisis General (CSV) con TOP 10
def analizar_csv_visual(archivo, hora_filtro):
    try:
        df = pd.read_csv(archivo.name)

        cols_modelo = ['aoi_type', 'courier_sla_rate', 'is_weekend', 'orders_by_courier',
                       'day_of_week', 'aoi_congestion', 'is_peak_hour', 'courier_mean_delay',
                       'courier_volume', 'distance_km', 'hour']

        # Rellenar columnas técnicas
        if 'is_weekend' not in df.columns: df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
        if 'is_peak_hour' not in df.columns: df['is_peak_hour'] = df['hour'].apply(lambda x: 1 if (13 <= x <= 15) or (20 <= x <= 22) else 0)
        if 'courier_sla_rate' not in df.columns: df['courier_sla_rate'] = 0.85
        if 'aoi_congestion' not in df.columns: df['aoi_congestion'] = df['hour'].apply(lambda x: 0.5 if (13 <= x <= 21) else 0.2)

        # --- PREDICCIÓN Y ASIGNACIÓN A "target_delay_min" ---
        df['target_delay_min'] = reg_model.predict(df[cols_modelo])

        # --- LÓGICA TOP 10 PEDIDOS CRÍTICOS ---
        if 'order_id' not in df.columns: df['order_id'] = [f"ORD-{i}" for i in range(len(df))]

        # Ordenamos por la columna de predicción: target_delay_min
        top_10_df = df.sort_values(by='target_delay_min', ascending=False).head(10).copy()
        top_10_df['target_delay_min'] = top_10_df['target_delay_min'].round(1)

        # Seleccionamos las columnas para la tabla de Gradio
        tabla_resumen = top_10_df[['order_id', 'hour', 'distance_km', 'target_delay_min']]

        # Histograma (usando target_delay_min)
        fig_hist = px.histogram(df, x="target_delay_min", color="aoi_type",
                               title="Distribución de Retrasos Estimados (Predicción)",
                               labels={'target_delay_min': 'Minutos de Retraso'},
                               barmode='overlay', opacity=0.7,
                               color_discrete_sequence=px.colors.qualitative.Safe)

        # Mapa de Calor
        df_mapa = df[df['hour'] == hora_filtro]
        if not df_mapa.empty:
            lat_center, lon_center = df_mapa['lat'].mean(), df_mapa['lng'].mean()
            m = folium.Map(location=[lat_center, lon_center], zoom_start=11, tiles='cartodbpositron')
            heat_data = [[row['lat'], row['lng'], row['target_delay_min']] for index, row in df_mapa.iterrows()]
            HeatMap(heat_data, radius=15, blur=10).add_to(m)
            mapa_html = m._repr_html_()
        else:
            mapa_html = "<h3>No hay datos para la hora seleccionada</h3>"

        return fig_hist, mapa_html, tabla_resumen

    except Exception as e:
        return None, f"<h3>Error: {e}</h3>", pd.DataFrame({"Error": [str(e)]})

# --- 3. INTERFAZ GRADIO ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚚 LaDe Analytics: Last Mile Pulse")
    gr.Markdown("Sistema inteligente de monitorización de latencia en entregas B2C.")

    with gr.Tab("🎯 Análisis Individual"):
        with gr.Row():
            with gr.Column():
                h_in = gr.Number(label="Hora (0-23)", value=14)
                d_in = gr.Number(label="Distancia (km)", value=5.0)
                aoi_in = gr.Dropdown(choices=[1, 2, 3, 4, 14], value=1, label="Tipo de Zona (AOI)")
                dw_in = gr.Dropdown(choices=["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"], value="Lunes", label="Día")
            with gr.Column():
                o_in = gr.Number(label="Pedidos asignados", value=10)
                md_in = gr.Number(label="Retraso Histórico (min)", value=15)
                vol_in = gr.Number(label="Volumen Total", value=100)

        btn_ind = gr.Button("⚡ CALCULAR PREDICCIÓN", variant="primary")

        with gr.Row():
            out_e = gr.Textbox(label="Estado de Riesgo")
            out_p = gr.Textbox(label="Probabilidad")
            out_m = gr.Textbox(label="Retraso Estimado")

        btn_ind.click(predecir_manual, [h_in, d_in, o_in, aoi_in, dw_in, md_in, vol_in], [out_e, out_p, out_m])

    with gr.Tab("📊 Dashboard de Operaciones"):
        gr.Markdown("### 🛠️ Carga Masiva de Operaciones")
        with gr.Row():
            file_input = gr.File(label="Sube archivo CSV de rutas")
            hour_map = gr.Slider(minimum=0, maximum=23, step=1, value=14, label="Filtrar Mapa por Hora")

        btn_gen = gr.Button("🔍 GENERAR DASHBOARD DE URGENCIAS", variant="primary")

        with gr.Row():
            plot_hist = gr.Plot(label="Distribución de Tiempos")

        gr.Markdown("### ⚠️ Top 10 Pedidos Críticos (Acción Inmediata)")
        tabla_retrasos = gr.Dataframe(
            headers=["ID Pedido", "Hora", "Distancia (km)", "Retraso Est. (min)"],
            interactive=False
        )

        gr.Markdown("#### 📍 Mapa de Calor: Concentración de Latencia")
        plot_map = gr.HTML()

        btn_gen.click(
            analizar_csv_visual,
            inputs=[file_input, hour_map],
            outputs=[plot_hist, plot_map, tabla_retrasos]
        )

demo.launch(debug=True, share=True)